In [1]:
from pyspark.sql.functions import regexp_replace, col, udf
from pyspark.sql.types import IntegerType, TimestampType
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
import plotly.graph_objects as go
import plotly
import datetime

In [2]:
def timestamp_transform(x):
  return datetime.datetime.fromtimestamp(x).hour
#spark.udf.register("timestamp_transform", timestamp_transform)
format_timestamp_udf = udf(lambda x: timestamp_transform(x))

In [3]:
def cat(cat_list, df):
  # The index of string vlaues multiple columns
  indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
    for c in cat_list
  ]
  
  # The encode of indexed vlaues multiple columns
  encoders = [OneHotEncoder(dropLast=True,inputCol=indexer.getOutputCol(),
              outputCol="{0}_encoded".format(indexer.getOutputCol())) 
      for indexer in indexers
  ]

  # Vectorizing encoded values
  assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders],outputCol="cat_features")

  pipeline = Pipeline(stages=indexers + encoders+[assembler])
  model=pipeline.fit(df)
  transformed = model.transform(df).select('cat_features')
  
  return transformed